In [ ]:
from IPython.display import clear_output

import import_ipynb
from plotter import Prompter
from table_abs import Abs
from table_trans import Trans
from gear_selection_lib import clear_pdnr_tuning, pre_pdnr_sweep, pdnr_sweep, set_trans_gears, set_abs_gears
from touchcomm import TouchComm

int_durs = list(range(24, 34))

test_conditions = [
    'White',
    'Blue',
    'Zebra 1',
    'White w/ charger noise',
    'home screen'
]

tc = None
num_gears = None
trans_table = None
trans_noise_data = []
abs_table = None
abs_noise_data = [[], [], []]

def reset_pdnr():
    clear_pdnr_tuning()

def do_pre_pdnr_sweep():
    pre_pdnr_sweep(tc, int_durs, num_gears)

def do_pdnr_sweep():
    noise_output = pdnr_sweep(tc, int_durs, num_gears)
    trans_noise_data.append(noise_output[0])
    abs_noise_data[0].append(noise_output[1])
    abs_noise_data[1].append(noise_output[2])
    abs_noise_data[2].append([*map(max, zip(noise_output[1], noise_output[2]))])

def generate_trans_table():
    global trans_table
    trans_table = Trans()
    trans_table.init_table(tc, int_durs, test_conditions, trans_noise_data, num_gears)
    trans_table.run()

def complete_trans_selection():
    set_trans_gears(tc, trans_table.get_selections(), num_gears)

def generate_abs_table():
    global abs_table
    abs_table = Abs()
    abs_table.init_table(tc, int_durs, test_conditions, abs_noise_data, num_gears)
    abs_table.run()

def complete_abs_selection():
    set_abs_gears(tc, abs_table.get_selections(), num_gears)

prompts = []
prompts.append('Pre-PDNR Sweep')
for condition in test_conditions:
    prompts.append('Please set up test condition "' + condition + '". Click Okay when done.')
prompts.append('PDNR Sweep')
for condition in test_conditions:
    prompts.append('Please set up test condition "' + condition + '". Click Okay when done.')
prompts.append('Gear Selection Table (Trans)')
prompts.append('Transcap gear selection completed.')
prompts.append('Gear Selection Table (Abs)')
prompts.append('Abs gear selection completed.')
prompts.append('End of gear selection process.')

callbacks = []
callbacks.append(reset_pdnr)
for condition in test_conditions:
    callbacks.append(do_pre_pdnr_sweep)
callbacks.append(None)
for condition in test_conditions:
    callbacks.append(do_pdnr_sweep)
callbacks.append(generate_trans_table)
callbacks.append(complete_trans_selection)
callbacks.append(generate_abs_table)
callbacks.append(complete_abs_selection)
callbacks.append(None)

try:
    tc = TouchComm.make('report_streamer')
    tc.getAppInfo()
    static = tc.getStaticConfig()
    num_gears = len(static['daqParams.freqTable[0].rstretchDur'])
    clear_output(wait=True)
    prompter = Prompter(mode='jupyterlab', height='150px', prompts=prompts, callbacks=callbacks, allow_back=False)
    prompter.run()
except Exception as e:
    print(e)
